In [2]:
# pip install pandas xlrd
import pandas as pd

import xlrd

url = 'https://www.sharkattackfile.net/spreadsheets/GSAF5.xls'
data_df = pd.read_excel(url)


In [17]:
import numpy as np

In [ ]:
#ORDENAR - esta es la importacion del archivo de las funciones de limpieza que me he creado
import sys
sys.path.append("SHARKATTACKSPROYECT/")
from src.functions import *

In [1]:
# We are taking a snapshot about our data panel in order to see to what we are facing.

data_df.shape()

In [ ]:
# It seems before step is enough to take a complete snapshot of our data. Lets go with another method.

pd.options.display.max_columns # We get access to configure display options from Pandas and configure max columns to show.

In [3]:
# Once we've seen the columns distribution, we gonna face the date types of each column to know how start to work with each one.
# We need to know how many Na/Non Values we have in data.
data_df.info()

In [ ]:
# Lets see to get a general idea about the problem with Na in order to clean approaching. We get percentages.

Na_amount = round(data_df.isna().sum()*100/len(data_df), 2)
Na_amount.sort_values(ascending = False)

In [11]:
# We see there are some columns with a too high percentage of Nan Values.
# A logical idea we can conclude, it is those columns with a high percentage of nan values are useless due to missinformation given.
# So We decide to drop columns named "Unnamed 21" and "Unnamed 22".





In [ ]:
# Eliminamos todas las filas con valores NA

data_df.dropna(axis=0, inplace= True, how='all')

df.dropna(axis=0, inplace= True, thresh=int(24*0.8))

In [ ]:
# Análisis de los datos de fecha

#Columna Date

#Vemos los valores únicos
data_df.Year.unique()

In [ ]:
# Tenemos valores muy raros, como año 0, 5, 77. Por cuestiones históricas, decidimos dropear todo lo anterior al año 1800.

# Como tenemos otra columna (date) que también nos da años, intentamos si podemos rescatar datos: 

data_df[['Date','Year']][data_df.Year == 0] #81 fechas que podemos rescatar a través de date

In [ ]:
data_df.Date = data_df.Date.replace(regex=r'(?i)Reported\s{1,9}',value='')
list(data_df.Date.unique())

In [18]:
# Aunque no salen en los uniques hay muchos Dates que salen como rangos, o datos de antes de Cristo.
# Vamos a denominarlos momentáneamente salvables para ver cuantos hay exactamente:
salvables = data_df.loc[(data_df["Year"] == 0) & (data_df["Date"] != np.nan)]
salvables.shape

(129, 23)

In [ ]:
data_df.Date.str.replace("-", " ") #1
data_df.Date.str.replace("_", " ")#2
data_df.Date.str.strip() #3
